In [17]:
import pandas as pd

In [18]:
df = pd.read_csv('train.csv')

In [19]:
df.head()

,train_idx,title,description,target,creator_name,creator_location,supporters,created_date,banner_image,full_content,victory_flag
0,0,Stop Iran Regime from Chairing U​.​N. Human Ri...,Rights Council 2023 Social Forum. This year's ...,['António Guterres'],UN Watch,"Geneva, Switzerland",80515,10 May 2023,https://assets.change.org/photos/2/ur/wj/vrUrW...,No Joke: The Islamic Republic of Iran has just...,0
1,1,Save Children In Poverty In Syria,Children in Syria are facing extreme poverty. ...,['UNICEF'],Maialen Alawam,"Watertown, MA, United States",23871,25 Jun 2020,https:https://static.change.org/images/default...,Children in Syria are facing extreme poverty. ...,0
2,2,Save Duke Ellington School Of The Arts,NaN,NaN,It’s Me Tivoni,United States,4051,"Nov. 20, 2022",https://assets.change.org/photos/8/ab/hv/DZaBH...,Save Duke Ellington School Of The ArtsTivoni H...,0
3,3,Mobile operators are creating inequality,"In India, after Reliance Jio (a 4G only mobile...","['Telecom Regulatory Authority of India', 'CEL...",Swastik Raj Chauhan,"Ghaziabad, India",350,"Oct 15, 2017",https://assets.change.org/photos/7/by/in/Kzbyi...,"In India, after Reliance Jio (a 4G only mobile...",0
4,4,Grant Military Burial Honors to Women WWII Pilots,"During World War II, a brave group of women jo...",['U.S. Senate'],Tiffany Miller @tiffbmiller,"Walnut Creek, CA, United States",176092,2 Dec 2015,https://assets.change.org/photos/3/lh/jc/TmLhJ...,"During World War II, a brave group of women jo...",1


In [ ]:
#Comprobación existencia valores nulos
df.isna().sum()
# Limpiar valores nulos:
df_purged=df.dropna()
df_purged.isna().sum()

In [ ]:
# Comprobamos si existen valores duplicados
df_purged.duplicated().sum()



In [ ]:
#  Crear una copia explícita del DataFrame
df_purged = df_purged.copy()

In [ ]:
# Eliminar data no relevante 
    #  Columna con la url de los banners
df_purged.drop('banner_image', axis=1, inplace=True)
    #  Columna nombre del creador
df_purged.drop('creator_name', axis=1, inplace=True)
    #  Columna fecha de creacion
df_purged.drop('created_date', axis=1, inplace=True) 
df_purged


DETECCION IDIOMAS PETICIONES

In [ ]:
%pip install nltk
%pip install langdetect
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('omw-1.4')
from langdetect import detect

def detect_language(text):
    # Tokenización de palabras
    tokens = word_tokenize(text.lower())

    # Lematización de palabras
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Detección del idioma
    language = detect(' '.join(tokens))
    return language


In [ ]:
# Creamos una columna donde guardamos el idioma de la peticion
df_purged['language'] = df_purged['title'].apply(detect_language)
df_purged

In [ ]:
# Lista de idiomas de las peticiones
df_purged['language'].unique()

In [ ]:
# Numerar los 5 idiomas con mayores peticiones
df_purged['language'].value_counts().head()

In [ ]:
# Obtener los 5 idiomas más usados
top_languages = df_purged['language'].value_counts().head(5).index.tolist()

# Filtrar el DataFrame para incluir solo las peticiones en los 5 idiomas más utilizados
df_filtered = df_purged[df_purged['language'].isin(top_languages)]

# Imprimir el DataFrame filtrado
(df_filtered)


In [ ]:
# Comprobación solidez de datos y consistencia
# Extraer solo el país de la columna 'creator_location'
df_filtered['creator_location'] = df_filtered['creator_location'].str.split(',').str[-1].str.strip()
df_filtered['creator_location'].value_counts()
df_filtered

In [ ]:
df_filtered.dtypes

## TRADUCIR AL INGLÉS LAS PETICIONES

In [ ]:
%pip install googletrans==3.1.0a0

In [ ]:
from googletrans import Translator
translator = Translator(service_urls=['translate.googleapis.com'])
import pandas as pd

# Definir una función para traducir una columna
def translate_column(column):
    translated_column = []
    for text in column:
        translation = translator.translate(text,dest='en').text
        translated_column.append(translation)
    return translated_column
# 
# Traducir la columna 'title' y almacenar los resultados en una nueva columna
df_filtered["title_translated"] = translate_column(df_filtered["title"])

# Traducir la columna 'description' y almacenar los resultados en una nueva columna
df_filtered["description_translated"] = translate_column(df_filtered["description"])

# Traducir la columna 'full_content' y almacenar los resultados en una nueva columna
df_filtered["full_content_translated"] = translate_column(df_filtered["full_content"])

# Mostrar el dataframe resultante
df_filtered


### ELIMINAR PALABRAS IRRELEVANTES (STOPWORDS)

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Descargar los stopwords si aún no están descargados
nltk.download('stopwords')

# Obtener los stopwords en el idioma deseado (en este caso, inglés)
stop_words = set(stopwords.words('english'))

# Definir una función para eliminar las palabras irrelevantes de un texto
def remove_stopwords(text):
    tokens = word_tokenize(text.upper())
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

# Aplicar la función remove_stopwords a la columna 'title_translated' del dataframe df_filtered
df_filtered['title_cleaned_nostopwords'] = df_filtered['title_translated'].apply(remove_stopwords)
df_filtered['description_nostopwords'] = df_filtered['description_translated'].apply(remove_stopwords)
df_filtered['full_content_nostopwords'] = df_filtered['full_content_translated'].apply(remove_stopwords)


# Mostrar el dataframe resultante
df_filtered


### CONTAR LAS PALABRAS MÁS REPETIDAS

#### CON LEMATIZACION

In [ ]:
# import nltk
# from nltk.stem import WordNetLemmatizer
# from collections import Counter

# # Descargar los recursos necesarios para nltk
# nltk.download('wordnet')

# # Inicializar el lematizador
# lemmatizer = WordNetLemmatizer()

# # Función para lematizar una lista de palabras
# def lemmatize_words(words):
#     return [lemmatizer.lemmatize(word) for word in words]

# # Obtener las palabras de la columna 'title_cleaned_nostopwords'
# # all_words = ' '.join(df_filtered['title_cleaned_nostopwords']).split()

# # Lematizar las palabras
# lemmatized_words = lemmatize_words(df_filtered['title_cleaned_nostopwords'])

# # Contar la frecuencia de cada palabra lematizada
# word_counts = Counter(lemmatized_words)

# # Obtener las palabras más repetidas y mostrarlas en orden descendente
# most_common_words = word_counts.most_common(20)

# # Mostrar las palabras más repetidas
# print('Palabras más repetidas (con lematización):')
# for word, count in most_common_words:
#     print(f'{word}: {count}')


#### SOLO PALABRAS MÁS UTILIZADAS

In [ ]:
import re
from collections import Counter

# Unir todas las palabras en una lista y eliminar los signos de puntuación
all_words = ' '.join(df_filtered['title_cleaned_nostopwords'])
all_words += ' '.join(df_filtered['description_nostopwords'])
all_words += ' '.join(df_filtered['full_content_nostopwords'])
all_words = re.findall(r'\b\w{2,}\b', all_words)

# Contar la frecuencia de cada palabra
word_counts = Counter(all_words)

# Obtener las palabras más repetidas y mostrarlas en orden descendente
most_common_words = word_counts.most_common(20)

# Mostrar las palabras más repetidas
print('Palabras más repetidas:')
for word, count in most_common_words:
    print(f'{word}: {count}')



# EXPORTAR BBDD CLEANED

In [63]:
# Crear una nueva DataFrame con las columnas específicas
df_clean_train = df_filtered[['title_cleaned_nostopwords', 'description_nostopwords', 'full_content_nostopwords', 'victory_flag', 'supporters']].copy()
df_clean_train.to_csv('dataframe_train.csv', index=False)
df_clean_train

,title_cleaned_nostopwords,description_nostopwords,full_content_nostopwords,victory_flag,supporters
0,STOP IRAN REGIME CHAIRING U​.​N . HUMAN RIGHTS...,RIGHTS COUNCIL 2023 SOCIAL FORUM . YEAR 'S THE...,JOKE : ISLAMIC REPUBLIC IRAN APPOINTED CHAIR U...,0,80515
1,SAVE CHILDREN POVERTY SYRIA,CHILDREN SYRIA FACING EXTREME POVERTY . OFTENT...,CHILDREN SYRIA FACING EXTREME POVERTY . OFTENT...,0,23871
3,MOBILE OPERATORS CREATING INEQUALITY,"INDIA , RELIANCE JIO ( 4G MOBILE OPERATOR ) , ...","INDIA , RELIANCE JIO ( 4G MOBILE OPERATOR ) , ...",0,350
4,GRANT MILITARY BURIAL HONORS WOMEN WWII PILOTS,"WORLD WAR II , BRAVE GROUP WOMEN JOINED WAR EF...","WORLD WAR II , BRAVE GROUP WOMEN JOINED WAR EF...",1,176092
5,N'T VIOLATE CHARTER RIGHTS,RULINGS FIND GOVERNMENT VIOLATED FUNDAMENTAL R...,ONTARIO PREMIER DOUG FORD FOLLOWING AUGUST 201...,0,36053
...,...,...,...,...,...
1960,JUSTICE POUR FORTUNE,TOGOLESE STUDENT MURDERED FRANCE,TOGOLESE STUDENT MURDERED FRANCE,1,15341
1961,100 % CLEAN RENEWABLE ENERGY 2030 .,"SWITCH 100 % CLEAN RENEWABLE ENERGY 2030 , CON...",LESS 7 YEARS EFFECTS CLIMATE CHANGE IRREVERSIB...,0,4036
1962,CREATE FAIR COMPASSIONATE UK IMMIGRATION POLICY,TREATED ? PETITION CHALLENGES U.K. GOVERNMENT ...,"DREAM LIVE ANOTHER COUNTRY , WOULD HOPE COUNTR...",0,6393
1963,APPROVE SOCIAL SECURITY DISABILITY,DISABILITY DENIED EVEN THOUGH MEDICAL HISTORY ...,WANT PERSONALLY THANK EVERY ONE SUPPORT DIFFIC...,1,71938


#### PLOTEAR IMAGEN CON PALABRAS MAS REPETIDAS

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Crear una wordcloud con las palabras más repetidas
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(dict(most_common_words))

# Mostrar la wordcloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()


## PARA MODELO 

In [69]:
# ATRIBUTOS MÁS IMPORTANTES
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Crear una matriz TF-IDF a partir de los campos de texto relevantes
tfidf = TfidfVectorizer()
text_features = tfidf.fit_transform(df_clean_train['full_content_nostopwords'])

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(text_features, df_clean_train['victory_flag'], test_size=0.2, random_state=42)

# Entrenar un modelo de regresión logística
model = LogisticRegression()
model.fit(X_train, y_train)

# Obtener la importancia de las características
feature_importances = model.coef_[0]

# Crear un DataFrame con las características y sus importancias
feature_df = pd.DataFrame({'Feature': tfidf.get_feature_names_out(), 'Importance': feature_importances})
feature_df = feature_df.sort_values('Importance', ascending=False)

# Imprimir las 10 características más importantes
top_features = feature_df.head(15)
print(top_features)

# Evaluar el modelo en el conjunto de prueba
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


       Feature  Importance
9183   company    0.878150
39512  surgery    0.877249
486       2013    0.867892
41083     trai    0.846458
25611  matthew    0.817679
19431     hens    0.816710
14065     eggs    0.752160
7067      cage    0.730241
43069  victims    0.720397
8467     cispa    0.681496
24198     like    0.665872
32950      put    0.649694
17145   france    0.634275
17739      gay    0.632598
33399     rape    0.629750
              precision    recall  f1-score   support

           0       0.86      1.00      0.92       285
           1       0.00      0.00      0.00        47

    accuracy                           0.86       332
   macro avg       0.43      0.50      0.46       332
weighted avg       0.74      0.86      0.79       332



/home/zuzu9627/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zuzu9627/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zuzu9627/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# OTRO MODELO

### Para utilizar las palabras más repetidas como atributos importantes en un modelo de predicción, puedes seguir los siguientes pasos:
Obtener las palabras más repetidas del DataFrame 'df_filtered' en la columna 'title_cleaned_nostopwords' utilizando el script que hemos modificado anteriormente:

In [44]:


# Obtener solo las palabras más repetidas (sin contar la frecuencia)
top_words = [word for word, _ in most_common_words]


Utilizar las palabras más repetidas como atributos importantes en el modelo de predicción. En lugar de utilizar el objeto TfidfVectorizer para extraer características de todo el texto, utilizaremos solo las palabras más repetidas.

In [60]:
from sklearn.feature_extraction.text import CountVectorizer

# Crear un vectorizador que solo incluya las palabras más repetidas
vectorizer = CountVectorizer(vocabulary=top_words)

# Obtener las características utilizando el vectorizador
text_features = vectorizer.transform(df_filtered['title_cleaned_nostopwords'])
text_features += vectorizer.transform(df_filtered['description_nostopwords'])
text_features += vectorizer.transform(df_filtered['full_content_nostopwords'])


# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(text_features, df_filtered['victory_flag'], test_size=0.2, random_state=42)

# Entrenar un modelo de regresión logística
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))




              precision    recall  f1-score   support

           0       0.86      1.00      0.92       285
           1       0.00      0.00      0.00        47

    accuracy                           0.86       332
   macro avg       0.43      0.50      0.46       332
weighted avg       0.74      0.86      0.79       332



/home/zuzu9627/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zuzu9627/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zuzu9627/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
